# Box Loader Testing

I need to test a few things about the steps in the `argo_box_loader` notebook. There are a few things that might make a difference by decreasing the size of an array, but I want to try here on a box that takes a lot less time to load.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw

import argopy
from argopy import DataFetcher as ArgoDataFetcher

argo_loader = ArgoDataFetcher(
    src="gdac", ftp="/swot/SUM05/dbalwada/Argo_sync", progress=True
)

In [2]:
import os
import sys
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
sys.path.append(os.path.abspath('../funcs'))

import processing_funcs as prf

In [ ]:
box = [-25,-23,0,1,0,2001]

In [ ]:
ds = argo_loader.region(box)

In [ ]:
ds

In [ ]:
ds.to_xarray()

Okay, this isn't incredibly helpful. Because it means I couldn't change the dtype of the data until after calling .to_xarray(), which is the step that uses up available memory. I'm now going to try loading multiple boxes in parallel using dask. Wish me luck

In [3]:
#box1 = [-180,-179,0,1,0,2001]
box2 = [-180,-179,1,2,0,2001]
box3 = [-180,-179,2,3,0,2001]
box4 = [-180,-179,3,4,0,2001]
box5 = [-180,-179,4,5,0,2001]

In [20]:
import dask
from dask import delayed
from dask.distributed import Client

# Initialize the Dask client
client = Client()

# Define your function with the delayed decorator
@dask.delayed
def get_box_delayed(*args, **kwargs):
    return prf.get_box(*args, **kwargs)

# Example arguments for your function
list_of_args = [
    #(box1,2),
    (box2,2),
    (box3,2),
    (box4,2),
    (box5,2)
]

# Create a list of delayed tasks
tasks = [get_box_delayed(*args) for args in list_of_args]

# Compute the results in parallel
results = dask.compute(*tasks)

# Process the results as needed
for n,result in enumerate(results):
    result.to_netcdf("/swot/SUM05/amf2288/sync-boxes/test_boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(list_of_args[n][0][0],list_of_args[n][0][1],list_of_args[n][0][2],list_of_args[n][0][3]))

/home.ufs/amf2288/mambaforge-pypy3/envs/Argo_Nov_24/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33659 instead
  warnings.warn(


loading points complete


  0%|          | 0/50 [00:00<?, ?it/s]

loading points complete


  0%|          | 0/39 [00:00<?, ?it/s]

loading points complete


  0%|          | 0/28 [00:00<?, ?it/s]

loading points complete


100%|██████████| 50/50 [06:23<00:00,  7.67s/it]


In [ ]:
from typing import List, Tuple

def generate_grid(min_lat: float, max_lat: float, min_lon: float, max_lon: float, step: float) -> List[Tuple[float, float, float, float]]:
    """Generates a grid of bounding boxes."""
    grid = []
    lat = min_lat
    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            box = (lat, lat + step, lon, lon + step)
            grid.append(box)
            lon += step
        lat += step
    return grid

def split_into_regions(min_lat: float, max_lat: float, min_lon: float, max_lon: float) -> List[Tuple[float, float, float, float]]:
    """Splits the target area into 20-degree by 20-degree regions."""
    return generate_grid(min_lat, max_lat, min_lon, max_lon, 20)

def split_into_boxes(region: Tuple[float, float, float, float]) -> List[Tuple[float, float, float, float]]:
    """Splits a 20-degree by 20-degree region into 5-degree by 5-degree boxes."""
    return generate_grid(region[0], region[1], region[2], region[3], 5)

def process_boxes(min_lat: float, max_lat: float, min_lon: float, max_lon: float, task_function):
    """Processes each 5-degree by 5-degree box with the specified task function."""
    regions = split_into_regions(min_lat, max_lat, min_lon, max_lon)
    for region in regions:
        boxes = split_into_boxes(region)
        for box in boxes:
            task_function(box)

# Example usage:
def example_task_function(box: Tuple[float, float, float, float]):
    """An example task function to demonstrate usage."""
    print(f"Processing box: {box}")

# Define the target area.
min_latitude = 20.0
max_latitude = 40.0
min_longitude = -100.0
max_longitude = -80.0

# Process the boxes with the example task function.
process_boxes(min_latitude, max_latitude, min_longitude, max_longitude, example_task_function)


In [ ]:
def generate_grid(min_lat: float, max_lat: float, min_lon: float, max_lon: float, step: float) -> List[Tuple[float, float, float, float]]:
    """Generates a grid of bounding boxes."""
    grid = []
    lat = min_lat
    while lat < max_lat:
        lon = min_lon
        while lon < max_lon:
            box = (lat, lat + step, lon, lon + step)
            grid.append(box)
            lon += step
        lat += step
    return grid

In [ ]:
generate_grid(0,90,0,90,5)

In [ ]:
def get_region_box():
    
    #split region into multiple boxes
    
    for box in boxes:
        get_box(region)

In [ ]:
def get_target_box():
    
    #split target into multiple regions
    
    for region in regions:
        get_region_box(region)
        

In [9]:
cleint.close()

NameError: name 'cleint' is not defined